# II. Simulate energy injection of your favorite model

`DM21cm` can take any energy injection of the form
$$\frac{\mathrm{d}N^{\gamma,e^\pm}}{\mathrm{d}E}(z, E)\cdot\Gamma\big(z, \delta(\vec x), T_k(\vec x), x_e(\vec x), \vec x\big)$$
i.e. redshift-dependent $\gamma$ and $e^\pm$ spectra, times a redshift & location-dependent injections rate $\Gamma$.

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
from astropy.cosmology import Planck18
import py21cmfast as p21c

from dm21cm.evolve import evolve
from dm21cm.utils import abscs

## 1. Building a custom `Injection` object
Let's build a dark matter $s$-wave annihilation `Injection` object, with the primary channel $\chi\chi\rightarrow\mu^-\mu^+$.

In [10]:
from dm21cm.injections.base import Injection
import dm21cm.physics as phys
from darkhistory.spec import pppc

import jax.numpy as jnp

To define an `Injection` object, one needs to define the following helper functions
- `__init__`
- `is_injecting_elec`
- `get_config`

and the injection functions

- `inj_rate`
- `inj_power`
- `inj_phot_spec`
- `inj_elec_spec`
- `inj_phot_spec_box`
- `inj_elec_spec_box`.

Note that detailed explanations of the `Injection` can be found in docstrings, such as:

In [4]:
Injection.inj_power?

Signature: Injection.inj_power(self, z_start, z_end=None, state=None)
Docstring:
Injection power density in [eV / pcm^3 s].
Used in DarkHistory. Assumes a homogeneous universe.

Args:
    z_start (float): Starting redshift of the redshift step of injection.
    z_end (float, optional): Ending redshift of the redshift step of injection. See details in inj_rate.
    state (dict, optional): State of the universe at z_start. Used for rates with feedback.

Returns:
    float: Injection power per average baryon in [eV / pcm^3 s].
File:      ~/dm21cm/DM21cm/dm21cm/injections/base.py
Type:      function

The s-wave annihilation injection event rate, as a function of overdensity $\delta$, can be expressed as
$$\frac{dN_\text{inj}}{dVdt}=\frac{1}{2}\langle\sigma v\rangle_s \left(\frac{\bar\rho(z)(1+\delta)}{m_\chi}\right)^2 (1+B(z,\delta)),$$
with each event injecting $2m_\chi$ of energy. $B(z, \delta)$ is a substructure boost factor.

In [12]:
class DMSWaveAnnihilationInjection (Injection):
    """Dark matter s-wave annihilation injection object.
    
    Args:
        primary (str): Primary injection channel. See darkhistory.pppc.get_pppc_spec
        m_DM (float): DM mass in [eV].
        sigmav (float, optional): Annihilation cross-section in [cm^3 / s].
    """

    def __init__(self, primary=None, m_DM=None, sigmav=None):
        self.mode = 'DM s-wave annihilation'
        self.primary = primary
        self.m_DM = m_DM
        self.sigmav = sigmav

        # specify fixed injection spectra
        self.phot_spec_per_inj = pppc.get_pppc_spec(
            self.m_DM, abscs['photE'], self.primary, 'phot', decay=False
        ) # [phot / inj] | Use darkhistory's pppc module to get spectra. The spectrum is in N (or dN/dE) per injection event.
        self.elec_spec_per_inj = pppc.get_pppc_spec(
            self.m_DM, abscs['elecEk'], self.primary, 'elec', decay=False
        ) # [elec / inj]

    def is_injecting_elec(self):
        # Required by DM21cm to decide whether to skip electron injections
        return not np.allclose(self.elec_spec_per_inj.N, 0.)
    
    def get_config(self):
        # Required by DM21cm's DarkHistory wrapper to check if cached solution has the correct injection.
        return {
            'mode': self.mode,
            'primary': self.primary,
            'm_DM': self.m_DM,
            'sigmav': self.sigmav
        }

    # Optionally add functions, for example:
    def boost_factor(self, z):
        """Your favorite boost factor model for the whole universe."""
        return 0. # No boost for now.

    def boost_factor_cell(self, z, delta):
        """Your favorite boost factor model for a default DM21cm cell (2 cMpc) with overdensity delta."""
        return 0. # No boost for now.

    #===== injections =====
    # Assuming Euler steps. z_end is not used.
    def inj_rate(self, z, z_end=None, **kwargs):
        # This function is used ONLY in DarkHistory, where the universe is assumed to be homogeneous.
        # One XX->YY is one event
        rho_DM = phys.rho_DM * (1+z)**3 # [eV / pcm^3] | pcm stands for physical cm
        n_DM = rho_DM / self.m_DM # [1 / pcm^3]
        return float( 1/2 * n_DM**2 * self.sigmav * (1+self.boost_factor(z)) ) # [inj / pcm^3 s]
    
    def inj_power(self, z, z_end=None, **kwargs):
        # This function is used ONLY in DarkHistory, where the universe is assumed to be homogeneous.
        return self.inj_rate(z) * 2 * self.m_DM # [eV / pcm^3 s]
    
    def inj_phot_spec(self, z, z_end=None, **kwargs):
        # This function is used ONLY in DarkHistory, where the universe is assumed to be homogeneous.
        return self.phot_spec_per_inj * self.inj_rate(z) # [phot / pcm^3 s]
    
    def inj_elec_spec(self, z, z_end=None, **kwargs):
        # This function is used ONLY in DarkHistory, where the universe is assumed to be homogeneous.
        return self.elec_spec_per_inj * self.inj_rate(z) # [elec / pcm^3 s]
    
    def inj_phot_spec_box(self, z, z_end=None, delta_plus_one_box=None, **kwargs):
        # This function is used ONLY in DM21cm.
        rho_DM_avg = phys.rho_DM * (1+z)**3 # [eV / pcm^3]
        n_DM_avg = rho_DM_avg / self.m_DM # [1 / pcm^3]
        inj_rate_box = 1/2 * (n_DM_avg * delta_plus_one_box)**2 * self.sigmav * (1 + self.boost_factor_cell(z, delta_plus_one_box-1))
        inj_rate_avg = jnp.mean(inj_rate_box)
        return self.inj_phot_spec(z) * float(inj_rate_avg), inj_rate_box / inj_rate_avg # [phot / pcm^3 s], [1]

    def inj_elec_spec_box(self, z, z_end=None, delta_plus_one_box=None, **kwargs):
        # This function is used ONLY in DM21cm.
        rho_DM_avg = phys.rho_DM * (1+z)**3 # [eV / pcm^3]
        n_DM_avg = rho_DM_avg / self.m_DM # [1 / pcm^3]
        inj_rate_box = 1/2 * (n_DM_avg * delta_plus_one_box)**2 * self.sigmav * (1 + self.boost_factor_cell(z, delta_plus_one_box-1))
        inj_rate_avg = jnp.mean(inj_rate_box)
        return self.inj_elec_spec(z) * float(inj_rate_avg), inj_rate_box / inj_rate_avg # [elec / pcm^3 s], [1]

## 2. After implementing the above methods, pass it into `evolve`

In [ ]:
return_dict = evolve(
    run_name = 'test2',
    z_start = 45.,
    z_end = 5.,
    injection = DMSWaveAnnihilationInjection(
        primary='mu', # XX-->mu+mu- channel
        m_DM=1e10, # [eV]
        sigmav=1e-30, # [cm^3 / s]
    ),
    p21c_initial_conditions = p21c.initial_conditions(
        user_params = p21c.UserParams(
            HII_DIM = 64,
            BOX_LEN = 256, # [conformal Mpc]
            N_THREADS = 32,
        ),
        cosmo_params = p21c.CosmoParams(
            OMm = Planck18.Om0,
            OMb = Planck18.Ob0,
            POWER_INDEX = Planck18.meta['n'],
            SIGMA_8 = Planck18.meta['sigma8'],
            hlittle = Planck18.h,
        ),
        random_seed = 12345,
        write = True,
    ),
    p21c_astro_params = p21c.AstroParams(),
)